In [1]:
!pip install transformers
!pip install accelerate
!pip install langchain
!pip install -U langchain-community
!pip install bitsandbytes
!pip install sentence_transformers
!pip install einops


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.4/412.4 kB 23.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 25.2 MB/s eta 0:00:00:00:0100:01


In [2]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 13.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 51.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 73.0 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4

In [3]:
!pip install jq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.4/737.4 kB 11.1 MB/s eta 0:00:00 0:00:01


In [4]:
from langchain.document_loaders import CSVLoader, JSONLoader
# converting documents into smaller chunks of token
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Embeddings
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings
# Storing and creating a persitance vectorized form of text
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from transformers import AutoTokenizer

import transformers
import torch
from torch import cuda, bfloat16

In [5]:
medication_loader = CSVLoader("/kaggle/input/medical-care-and-chats/format_dataset.csv")
medical_document = medication_loader.load()

In [6]:
display(medical_document[0:10])

[Document(metadata={'source': '/kaggle/input/medical-care-and-chats/format_dataset.csv', 'row': 0}, page_content="idx: 0\ndisease: Panic disorder\nSymptom: ['Anxiety and nervousness', 'Depression', 'Shortness of breath', 'Depressive or psychotic symptoms', 'Sharp chest pain', 'Dizziness', 'Insomnia', 'Abnormal involuntary movements', 'Chest tightness', 'Palpitations', 'Irregular heartbeat', 'Breathing fast']\nreason: Panic disorder is an anxiety disorder characterized by recurring severe panic attacks. It may also include significant behavioral changes lasting at least a month and of ongoing worry about the implications or concern about having other attacks. The latter are called anticipatory attacks (DSM-IVR). Panic disorder is not the same as agoraphobia (fear of public places), although many afflicted with panic disorder also suffer from agoraphobia. Panic attacks cannot be predicted, therefore an individual may become stressed, anxious or worried wondering when the next panic attac

In [7]:
jq_schema = ".[] | {instruction: .instruction, input: .input, output: .output}"

chats_json_loader = JSONLoader("/kaggle/input/medical-care-and-chats/chatdoctor5k.json", jq_schema = jq_schema, text_content=False)
doctor_chats = chats_json_loader.load()

In [8]:
doctor_chats[0:10]

[Document(metadata={'source': '/kaggle/input/medical-care-and-chats/chatdoctor5k.json', 'seq_num': 1}, page_content='{"instruction": "If you are a doctor, please answer the medical questions based on the patient\'s description.", "input": "Doctor, I have been experiencing sudden and frequent panic attacks. I don\'t know what to do.", "output": "Well, based on what you\'re telling me, it sounds like you may be suffering from panic disorder. The best course of action is to start with psychotherapy and mental health counseling. Additionally, we should conduct an electrocardiogram to make sure that there are no physical issues causing your panic attacks. We will also need to perform a depression screen and a toxicology screen to rule out any other underlying causes. Finally, I would recommend a comprehensive psychological and psychiatric evaluation and therapy to help manage your symptoms."}'),
 Document(metadata={'source': '/kaggle/input/medical-care-and-chats/chatdoctor5k.json', 'seq_num

In [9]:
medical_document_full = medical_document + doctor_chats

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 400, chunk_overlap = 50)
texts = text_splitter.split_documents(medical_document_full)

In [11]:
#https://huggingface.co/settings/tokens for hugging face key
import os 

os.environ['HuggingFaceHub_API_Token'] = ""

In [12]:
embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")

<ipython-input-12-e130fb7d776b>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
vectordb = Chroma.from_documents(
    documents = texts,
    embedding = embeddings,
    persist_directory = "Medication_Dir"
)

vectordb.persist()

<ipython-input-13-de3a51b5c5d8>:7: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [15]:
model_id = "/kaggle/input/llama-2/pytorch/7b-chat-hf/1"

# set quantization configuration to load large model with less GPU memory this requires the `bitsandbytes` library

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [16]:
model_config = transformers.AutoConfig.from_pretrained(
    model_id
)

In [17]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

In [19]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [20]:
query_pipeline = transformers.pipeline("text-generation", model=model, tokenizer=tokenizer, torch_dtype=torch.float16,
                            device_map="auto")

Device set to use cuda:0


In [21]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline = query_pipeline)

<ipython-input-21-e300255b25d0>:3: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline = query_pipeline)


In [22]:
Retrieval_Chain = RetrievalQA.from_chain_type(llm,
                           retriever = vectordb.as_retriever(),
                           return_source_documents = True)

In [23]:
result = Retrieval_Chain("I am having headache, cold and high fever. Suggest me medication.")

<ipython-input-23-d5ebcd0f2b88>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = Retrieval_Chain("I am having headache, cold and high fever. Suggest me medication.")
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [24]:
print(result['result'])

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

to help. You'll need to take Ibuprofen for your fever and body aches. We'll also give you some antibiotics such as Troleandomycin and Grepafloxacin to treat the underlying infection."}

Raloxifene (Evista), Sitagliptin (Januvia), Hyzaar and Terazosin as prescribed by me based on your condition."}

{"instruction": "If you are a doctor, please answer the medical questions based on the patient's description.", "input": "Doctor, I have been experiencing severe headaches and a high fever. I also feel really nauseous and dizzy. What could be wrong with me?", "output": "Based on your symptoms, it's possible that you have meningitis. To treat this infection, you will need to take a combination of

{"instruction": "If you are a doctor, please answer the medical questions based on the patient's description.", "input": "Doctor, I have 